<a href="https://colab.research.google.com/github/Bakison/bonthe-island1955/blob/main/image_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import tensorflow as tf
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.preprocessing.image import array_to_img
from IPython.display import display

In [ ]:
from numpy.random import seed
seed(888)
tf.random.set_seed(404)

In [ ]:
(x_train_all, y_train_all), (x_test, y_test) = cifar10.load_data()


In [ ]:
type(cifar10)

In [ ]:
from IPython.display import display
from keras.preprocessing.image import array_to_img
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

Constants

In [ ]:
LABEL_NAMES =['plane','car','Bird','cat','Deer','Dog','Frog','Horse','Ship',"Truck"]
IMAGE_WIDTH = 32
IMAGE_HEIGHT = 32
IMAGE_PIXELS = IMAGE_WIDTH * IMAGE_HEIGHT
COLOR_CHANNELS = 3
TOTAL_INPUTS = IMAGE_PIXELS * COLOR_CHANNELS

VALIDATION_SIZE = 10000
SMALL_TRAIN_SIZE = 1000

In [ ]:
x_train_all[0]

In [ ]:
pic = array_to_img(x_train_all[7])
display(pic)

In [ ]:
y_train_all.shape

In [ ]:
y_train_all[7][0]

In [ ]:
LABEL_NAMES[y_train_all[7][0]]

In [ ]:
plt.imshow(x_train_all[14])
plt.xlabel(LABEL_NAMES[y_train_all[4][0]], fontsize = 15)
plt.show()

In [ ]:
plt.figure(figsize = (15,5))
for i in range(20):
  plt.subplot(1, 20, i+1)
  plt.yticks([])
  plt.xticks([])
  plt.xlabel(LABEL_NAMES[y_train_all[i][0]], fontsize = 15)
  plt.imshow(x_train_all[i])

In [ ]:
x_train_all[0].shape

In [ ]:
nr_images, x, y, c = x_train_all.shape
print(f'images={nr_images} \t | width = {x} \t | heighr = {y} \t | channels  = {c}')

In [ ]:
x_test.shape

Proprocessing the Image DataSet

In [ ]:
x_train_all[0]

In [ ]:
type(x_train_all[0][0][0][0])

In [ ]:
x_train_all, x_test_all = x_train_all / 255.0, x_test / 255.0

In [ ]:
x_train_all[0][0][0][0]

In [ ]:
x_train_all = x_train_all.reshape(x_train_all.shape[0], TOTAL_INPUTS)

In [ ]:
x_train_all.shape

In [ ]:
x_test_all = x_test_all.reshape(x_test_all.shape[0], TOTAL_INPUTS)
print(f'shape of x_test is {x_test.shape}')

CREATE VALIDATION DATASET

In [ ]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]
x_val.shape
#y_val.shape

In [ ]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]
x_train.shape

In [ ]:
y_train.shape

Creating a small dataset(For illustration purposes)


In [ ]:
x_train_xs = x_train[:SMALL_TRAIN_SIZE]
y_train_xs = y_train[:SMALL_TRAIN_SIZE]

**Definding the Neural Network Using Keras**

In [ ]:
model_1 =  Sequential([
    Dense(units=128, input_dim = TOTAL_INPUTS, activation='relu', name = 'ml_hidden'),
    Dense(units=64, activation='relu', name = 'm2_hidden'),
    Dense(16, activation= 'relu', name = 'm3_hidden'),
    Dense(10, activation='softmax')
])
model_1.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

In [ ]:
type(model_1)

In [ ]:
model_1.summary()